In [1]:
def date_posted(days) -> str:
    return "r" + str(days * 86400)

date_posted(7)

'r604800'

In [3]:
from pydantic import BaseModel,Field,computed_field
from typing import List
from enum import Enum

In [4]:
class JobType(str,Enum):
    REMOTE = 'remote'
    HYBRID = 'hybrid'
    ONSITE = 'onsite'

class ContractType(str, Enum):
    FULL_TIME = "Full_time"
    PART_TIME = "Part_time"
    CONTRACT = "Contract"
    TEMPORARY = "Temporary"
    INTERNSHIP = "Internship"
    APPRENTICESHIP = "Apprenticeship"

In [5]:
class JobInfo(BaseModel):
    title : str | None = Field(default=None,description= "Primary job title or role to search for.This represents the main occupation or position of interest." ,examples=['AI engineer','Data Scientist','SQL','Java','Software Engineer'])
    location : str | None = Field(default=None,description= "The place where the job needs to be find",examples=['India','Banglore','America','Los Angeles','Rajasthan'])
    days : int | None = Field(default=7,description= "Job posted within the last days ",examples=[1,3,7,14])
    company_name : List[str] | None = Field(default=None,description= "The List of companies which needs to consider first or which is only needs to considered",examples=['Google','Microsoft'])
    companyId : List[str] | None = Field(default=None,description= "The List of Ids of the companies which needs to consider first or which is only needs to considered",examples=['21345','5567483'])
    skipJobId : List[str] | None = Field(default=None,description= "The List of Ids of the companies which needs to  be skiped or not considered",examples=['21345','5567483'])
    jobType : List[JobType] | None = Field(default=None,description= "This is the list of type of job the user preferred rankwise",examples=[['remote','hybrid'],['onsite']])
    experienceLevel : List[str] | None = Field(default=None,description= "This is the experience level needed for the job preferred by the user in years rankwise",examples=['1','2','3'])
    typeOfContract : List[ContractType] | None = Field(default=None,description= "The type of the ontract preffered by the user rankwise ",examples=[['Part_time','Full_time']])
    limit : int | None = Field(default=3,description= "The number of jobs the user wants to find even if the user will say a big number limit it up to 3",le=3,ge=1,examples=[3,1,2])

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite')

In [ ]:
user_prompt = "I want to find the jobs for AI engineering remote internship find 100 jobs in Ahmedabad"


In [24]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(template="""
        You are job searching helping agent you need to pass the 
        data to job searchng API for that you need to structure the 
        output from the given user prompt : {user_prompt}
        """,
        input_variables=['user_prompt'])

In [ ]:
job_info_agent = (
    prompt | 
    llm.with_structured_output(JobInfo)
    )

In [ ]:
res = job_info_agent.invoke({'user_prompt':user_prompt})

JobInfo(title='AI engineer', location='Ahmedabad', days=7, company_name=None, companyId=None, skipJobId=None, jobType=[<JobType.REMOTE: 'remote'>], experienceLevel=None, typeOfContract=[<ContractType.INTERNSHIP: 'Internship'>], limit=3)